# Controllable generation via RL to let Elon Musk speak ill of DOGE
> How to control text generation through a sentiment classifier.



In [ ]:
%pip install pfrl@git+https://github.com/voidful/pfrl.git
%pip install textrl==0.2.15

In [ ]:
from textrl import TextRLEnv,TextRLActor
from transformers import pipeline, AutoModelForTokenClassification, AutoTokenizer, AutoModelWithLMHead
import logging
import sys
import pfrl
import torch
logging.basicConfig(level=logging.INFO, stream=sys.stdout, format='')

**Using a pre-trained model, it can generate elonmusk's style tweets.**

In [ ]:
from transformers import AutoTokenizer
# tokenizer = AutoTokenizer.from_pretrained("lca0503/speech-chatgpt-base-nar-v2-epoch4-wotrans")
# model = AutoModelForSeq2SeqLM.from_pretrained("lca0503/speech-chatgpt-base-nar-v2-epoch4-wotrans")

tokenizer = AutoTokenizer.from_pretrained("huggingtweets/elonmusk")  
model = AutoModelWithLMHead.from_pretrained("huggingtweets/elonmusk")
model.eval()
model.cuda()

**a sentiment classifier for rl reward**

In [ ]:
sentiment = pipeline('sentiment-analysis',model="cardiffnlp/twitter-roberta-base-sentiment",tokenizer="cardiffnlp/twitter-roberta-base-sentiment",device=0,return_all_scores=True)

In [ ]:
transformers_logger = logging.getLogger('transformers')
transformers_logger.setLevel(logging.CRITICAL)

In [ ]:
sentiment("dogecoin is bad")

In [22]:
sentiment("dogecoin is bad")[0][0]['score']

0.9338533878326416

set our text generation reward, inverse perplexity + sentiment classifier.
- inverse perplexity make sure the generated sentence probability will be high.
- sentiment classifier can make the generate more negative.

In [ ]:
class MyRLEnv(TextRLEnv):
    def get_reward(self, input_item, predicted_list, finish): # predicted will be the list of predicted token
      reward = 0
      if finish or len(predicted_list) >= self.env_max_length:
        predicted_text = tokenizer.convert_tokens_to_string(predicted_list[0])
        # sentiment classifier
        reward = sentiment(input_item[0]+predicted_text)[0][0]['score'] * 10
      return reward

**fit one example**

In [ ]:
observaton_list = [{'input':'i think dogecoin is'}]

In [ ]:
env = MyRLEnv(model, tokenizer, observation_input=observaton_list,compare_sample=1)
actor = TextRLActor(env,model,tokenizer)
agent = actor.agent_ppo(update_interval=100, minibatch_size=3, epochs=10)

In [23]:
actor.predict(observaton_list[0])

KeyError: 0

In [ ]:
pfrl.experiments.train_agent_with_evaluation(
    agent,
    env,
    steps=300,
    eval_n_steps=None,
    eval_n_episodes=1,       
    train_max_episode_len=100,  
    eval_interval=10,
    outdir='elon_musk_dogecoin', 
)

loading the best result and predict.

In [ ]:
agent.load("./elon_musk_dogecoin/best")

In [ ]:
actor.predict(observaton_list[0])